# Netflix Content Strategy

### Project Title  
What types of content are most commonly released on the platform and how has that changed over time?
A data-driven approach for strategic content acquisition planning

### Business Model  
Subscription-based revenue  
User acquisition and retention through relevant content


### Business Objective  

Identify the content trends in terms of genres, format (movies vs. TV shows), and release patterns over time, in order to optimize catalog expansion and investment strategies

###  Hypothesis & Research Questions

#### Hypothesis:
Content characteristics such as genre, format, and release year are key factors influencing Netflix’s content planning and catalog composition over time.

#### Research Questions:
1.  Which genres are most frequently released on the platform?
2.	How has the composition of content types (Movies vs. TV Shows) changed over the years?
3.	What are the dominant trends in release years, and how consistent is content output over time?
4.	Which content durations (e.g. short films, standard, or long-format content) are most common?


These questions will guide the EDA and help shape strategic recommendations for Netflix’s content acquisition and promotion strategy.

In [25]:
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

df = pd.read_csv("netflix_titles.csv")

df.head()



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [26]:
# Count missing values
print(df.isnull().sum())

# View rows with missing values
print(df[df.isnull().any(axis=1)])

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64
     show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water              NaN   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans              NaN   
4         s5  TV Show           Kota Factory              NaN   
...      ...      ...                    ...              ...   
8795   s8796  TV Show        Yu-Gi-Oh! Arc-V              NaN   
8796   s8797  TV Show             Yunus Emre              NaN   
8797   s8798  TV Show              Zak Storm              NaN   
8800   s8801  TV Show     Zindagi Gulzar Hai              NaN   
8803   s8804  TV Sho

In [27]:
# Count non-ASCII characters
non_ascii = df.applymap(lambda x: any(ord(c) > 127 for c in str(x)) if pd.notnull(x) else False)
print(non_ascii.sum())

show_id            0
type               0
title            223
director         355
cast            1324
country            0
date_added         0
release_year       0
rating             0
duration           0
listed_in          0
description     1297
dtype: int64


/var/folders/9p/vp93bjn92s7ggp8mhnv9r3nw0000gn/T/ipykernel_27236/1480794867.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  non_ascii = df.applymap(lambda x: any(ord(c) > 127 for c in str(x)) if pd.notnull(x) else False)


In [28]:
def remove_non_ascii(text):
    if pd.isnull(text):
        return text
    return ''.join([i if ord(i) < 128 else '' for i in str(text)])

df_clean = df.applymap(remove_non_ascii)

/var/folders/9p/vp93bjn92s7ggp8mhnv9r3nw0000gn/T/ipykernel_27236/2514250213.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df.applymap(remove_non_ascii)


In [29]:
df_clean = df.fillna("Unknown")

In [30]:
df_clean = df_clean.fillna('') 

In [31]:
df_clean['description'].str.len().max()

np.int64(248)

In [32]:
print(df_clean['release_year'].unique())

[2020 2021 1993 2018 1996 1998 1997 2010 2013 2017 1975 1978 1983 1987
 2012 2001 2014 2002 2003 2004 2011 2008 2009 2007 2005 2006 1994 2015
 2019 2016 1982 1989 1990 1991 1999 1986 1992 1984 1980 1961 2000 1995
 1985 1976 1959 1988 1981 1972 1964 1945 1954 1979 1958 1956 1963 1970
 1973 1925 1974 1960 1966 1971 1962 1969 1977 1967 1968 1965 1946 1942
 1955 1944 1947 1943]


In [33]:
df_clean['release_year'] = pd.to_numeric(df_clean['release_year'], errors='coerce').fillna(0).astype(int)

In [34]:
df.rename(columns={
    'show_id': 'Show ID',
    'type': 'Type',
    'title': 'Title',
    'director': 'Director',
    'cast': 'Cast',  
    'country': 'Country',
    'date_added': 'Date Added',
    'release_year': 'Release Year',
    'rating': 'Rating',
    'duration': 'Duration',
    'listed_in': 'Genres',
    'description': 'Description'
}, inplace=True)

In [35]:
df_clean.to_csv('netflix_titles_clean.csv', index=False, encoding='utf-8')

In [36]:
df.head()

,Show ID,Type,Title,Director,Cast,Country,Date Added,Release Year,Rating,Duration,Genres,Description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [37]:
# Count duplicates before removal
print("Duplicate rows before:", df.duplicated().sum())

# Remove duplicates
df.drop_duplicates(inplace=True)

# Count duplicates after
print("Duplicate rows after:", df.duplicated().sum())

Duplicate rows before: 0
Duplicate rows after: 0


In [38]:
# Check missing values before filling
print("Missing values before:")
print(df[['Country', 'Director', 'Cast']].isnull().sum())

# Fill missing values
df['Country'] = df['Country'].fillna("Unknown")
df['Director'] = df['Director'].fillna("Not specified")
df['Cast'] = df['Cast'].fillna("Not specified")

# Check missing values after filling
print("\nMissing values after:")
print(df[['Country', 'Director', 'Cast']].isnull().sum())

Missing values before:
Country      831
Director    2634
Cast         825
dtype: int64

Missing values after:
Country     0
Director    0
Cast        0
dtype: int64


In [39]:
# Check non-datetime rows before conversion
print("Non-null before conversion:", df['date_added'].notnull().sum())
print("Trying to convert...")

# Convert column
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Show number of successfully converted values
print("Non-null after conversion:", df['date_added'].notnull().sum())

KeyError: 'date_added'

In [ ]:
# Number of rows before deletion
print("Rows before removing invalid dates:", len(df))

# Remove rows with NaT
df = df[df['date_added'].notnull()]

# Number of rows after
print("Rows after removing invalid dates:", len(df))

Rows before removing invalid dates: 8807
Rows after removing invalid dates: 8709


In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print("Categorical columns:\n", categorical_cols)

# Preview unique values in each categorical column
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(dropna=False).head(10))  

# Clean categorical columns: remove extra spaces and unify case 
for col in categorical_cols:
    df[col] = df[col].astype(str).str.strip()  
    df[col] = df[col].replace('nan', pd.NA)    

Categorical columns:
 Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'rating',
       'duration', 'listed_in', 'description'],
      dtype='object')

Column: show_id
show_id
s1       1
s5810    1
s5804    1
s5805    1
s5806    1
s5807    1
s5808    1
s5809    1
s5811    1
s5445    1
Name: count, dtype: int64

Column: type
type
Movie      6131
TV Show    2578
Name: count, dtype: int64

Column: title
title
Dick Johnson Is Dead                                              1
XOXO                                                              1
Forensic Files                                                    1
Saudi Arabia Uncovered                                            1
The Fierce Wife                                                   1
Witches: A Century of Murder                                      1
Glitter Force                                                     1
Jeff Foxworthy and Larry the Cable Guy: We’ve Been Thinking...    1
I'll Sleep When I'm Dead     

In [ ]:
# Convert duration to string (if it's not already)
df['duration'] = df['duration'].astype(str)

# Extract numeric part of duration
df['duration_int'] = df['duration'].str.extract(r'(\d+)').astype(float)

# Extract text part (type: min, Season, etc.)
df['duration_type'] = df['duration'].str.extract(r'([a-zA-Z]+)')

# Show some sample results
print(df[['duration', 'duration_int', 'duration_type']].head(10))

    duration  duration_int duration_type
0     90 min          90.0           min
1  2 Seasons           2.0       Seasons
2   1 Season           1.0        Season
3   1 Season           1.0        Season
4  2 Seasons           2.0       Seasons
5   1 Season           1.0        Season
6     91 min          91.0           min
7    125 min         125.0           min
8  9 Seasons           9.0       Seasons
9    104 min         104.0           min


In [ ]:
# Find top 10 countries by frequency
top_countries = df['country'].value_counts().nlargest(10).index

# Replace all other countries with 'Other'
df['country'] = df['country'].apply(lambda x: x if x in top_countries else 'Other')

# Show value counts after replacement
print(df['country'].value_counts())

country
United States     2778
Other             2748
India              971
Unknown            827
United Kingdom     403
Japan              241
South Korea        195
Canada             173
Spain              141
France             122
Mexico             110
Name: count, dtype: int64


In [ ]:
# Clean rating column (remove spaces, make uppercase)
df['rating'] = df['rating'].astype(str).str.strip().str.upper()

# Normalize similar categories
df['rating'] = df['rating'].replace({
    'NOT RATED': 'UNRATED',
    'NR': 'UNRATED',
    'UR': 'UNRATED',
    'NONE': 'UNRATED',
    'TV-M': 'TV-MA',
    'TV14': 'TV-14',
    'TVPG': 'TV-PG'
})

# Replace 'nan' strings and real NaNs with 'UNKNOWN'
df['rating'] = df['rating'].replace('NAN', pd.NA)
df['rating'] = df['rating'].fillna('UNKNOWN')

# Show resulting value counts
print(df['rating'].value_counts())

rating
TV-MA       3183
TV-14       2133
TV-PG        838
R            799
PG-13        490
TV-Y7        330
TV-Y         300
PG           287
TV-G         212
UNRATED       81
G             41
TV-Y7-FV       5
<NA>           4
NC-17          3
74 MIN         1
84 MIN         1
66 MIN         1
Name: count, dtype: int64


In [ ]:
# Remove rows where 'rating' contains values like 'MIN' (which likely belong to the 'duration' column)
df = df[~df['rating'].str.contains('MIN', na=False)]

# Display the cleaned rating value counts
print(df['rating'].value_counts())

rating
TV-MA       3183
TV-14       2133
TV-PG        838
R            799
PG-13        490
TV-Y7        330
TV-Y         300
PG           287
TV-G         212
UNRATED       81
G             41
TV-Y7-FV       5
<NA>           4
NC-17          3
Name: count, dtype: int64


In [ ]:
# Number of rows before cleaning duration_int
print("Rows before cleaning duration_int:", len(df))

# Remove rows where duration_int is NaN or 0
df = df[df['duration_int'].notna()]
df = df[df['duration_int'] != 0]

# Number of rows after cleaning
print("Rows after cleaning duration_int:", len(df))

# Check a few examples to confirm
print(df[['duration', 'duration_int', 'duration_type']].head(10))

Rows before cleaning duration_int: 8706
Rows after cleaning duration_int: 8706
    duration  duration_int duration_type
0     90 min          90.0           min
1  2 Seasons           2.0       Seasons
2   1 Season           1.0        Season
3   1 Season           1.0        Season
4  2 Seasons           2.0       Seasons
5   1 Season           1.0        Season
6     91 min          91.0           min
7    125 min         125.0           min
8  9 Seasons           9.0       Seasons
9    104 min         104.0           min


In [ ]:
# Convert 'date_added' to datetime format
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Check number of rows before and after removing invalid dates
print("Rows before removing invalid dates:", len(df))
df = df[df['date_added'].notna()]
print("Rows after removing invalid dates:", len(df))

# Show some cleaned date values
print(df['date_added'].sort_values().head(10))

Rows before removing invalid dates: 8706
Rows after removing invalid dates: 8706
5957   2008-01-01
6611   2008-02-04
5956   2009-05-05
5955   2009-11-18
7370   2010-11-01
5954   2011-05-17
5953   2011-09-27
5942   2011-10-01
5945   2011-10-01
5946   2011-10-01
Name: date_added, dtype: datetime64[ns]


In [ ]:
# Convert 'date_added' to datetime format
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Show sample and data type
print(df['date_added'].head())
print("\nData type of 'date_added':", df['date_added'].dtype)

0   2021-09-25
1   2021-09-24
2   2021-09-24
3   2021-09-24
4   2021-09-24
Name: date_added, dtype: datetime64[ns]

Data type of 'date_added': datetime64[ns]


In [ ]:
# Extract year and month from 'date_added'
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

# Show some sample results
print(df[['date_added', 'year_added', 'month_added']].head())

  date_added  year_added  month_added
0 2021-09-25        2021            9
1 2021-09-24        2021            9
2 2021-09-24        2021            9
3 2021-09-24        2021            9
4 2021-09-24        2021            9


In [ ]:
# Remove extra spaces around genre names
df['listed_in'] = df['listed_in'].astype(str).apply(lambda x: ', '.join([genre.strip() for genre in x.split(',')]))

# Optionally: convert genres into lists (for future analysis)
df['genre_list'] = df['listed_in'].apply(lambda x: [genre.strip() for genre in x.split(',')])

# Show sample output
print(df[['listed_in', 'genre_list']].head())

                                           listed_in  \
0                                      Documentaries   
1    International TV Shows, TV Dramas, TV Mysteries   
2  Crime TV Shows, International TV Shows, TV Act...   
3                             Docuseries, Reality TV   
4  International TV Shows, Romantic TV Shows, TV ...   

                                          genre_list  
0                                    [Documentaries]  
1  [International TV Shows, TV Dramas, TV Mysteries]  
2  [Crime TV Shows, International TV Shows, TV Ac...  
3                           [Docuseries, Reality TV]  
4  [International TV Shows, Romantic TV Shows, TV...  


In [ ]:
#  Explode the genre list
genre_exploded = df.explode('genre_list')

# Count the most common genres
top_genres = genre_exploded['genre_list'].value_counts().head(10)

# Show the result
print("Top 10 most common genres:")
print(top_genres)

Top 10 most common genres:
genre_list
International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1328
Documentaries                869
Action & Adventure           859
Independent Movies           756
TV Dramas                    739
Children & Family Movies     641
Romantic Movies              616
Name: count, dtype: int64


In [ ]:
#Strip leading/trailing whitespace
df['description'] = df['description'].astype(str).str.strip()

# Replace missing or empty descriptions
df['description'] = df['description'].replace(['', 'nan', 'NaN'], pd.NA)
df['description'] = df['description'].fillna('No description available')



In [ ]:
# Check data types of all columns
print(df.dtypes)

show_id                  object
type                     object
title                    object
director                 object
cast                     object
country                  object
date_added       datetime64[ns]
release_year              int64
rating                   object
duration                 object
listed_in                object
description              object
duration_int            float64
duration_type            object
year_added                int32
month_added               int32
genre_list               object
dtype: object


In [ ]:
# Convert only object columns with string values to category type
for col in df.select_dtypes(include='object').columns:
    try:
      
        if df[col].apply(lambda x: isinstance(x, str) or pd.isnull(x)).all():
            df[col] = df[col].astype('category')
        else:
            print(f"Skipping column {col}: contains non-string values (e.g., lists)")
    except Exception as e:
        print(f"Skipping column {col} due to: {e}")

Skipping column genre_list due to: setting an array element with a sequence.


In [ ]:
print(df.dtypes)

show_id                category
type                   category
title                  category
director               category
cast                   category
country                category
date_added       datetime64[ns]
release_year              int64
rating                 category
duration               category
listed_in              category
description            category
duration_int            float64
duration_type          category
year_added                int32
month_added               int32
genre_list               object
dtype: object


In [ ]:
# Final check for missing values
print(df.isna().sum())

show_id          0
type             0
title            0
director         0
cast             0
country          0
date_added       0
release_year     0
rating           0
duration         0
listed_in        0
description      0
duration_int     0
duration_type    0
year_added       0
month_added      0
genre_list       0
dtype: int64


In [ ]:
pip install pandas sqlalchemy mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

csv_path = '/Users/viktoria170/Desktop/Ironhack/Final project/netflix-content-strategy/netflix_titles.csv'
df = pd.read_csv(csv_path)

username = "root"
password = "Babolat575"  
host = "localhost"
database = "netflix"

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}")

df.to_sql(name="titles", con=engine, if_exists='replace', index=False)

print(" MySQL.")

 MySQL.


In [41]:
query = "SELECT * FROM titles LIMIT 20"
pd.read_sql(query, con=engine)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",None,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",None,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
